# Task 2 - Demand Analysis

In [1]:
# Load necessay packages
import pandas as pd 
import seaborn as sb
import numpy as np

In [2]:
# Load dataset
demand_data = pd.read_excel('C:\\Users\\Yuri_Almeida\\Desktop\\NOKIA_ANALYSIS\\Task2\\Task_2.xlsx', sheet_name='2.task')
# Data Preparation 
# Retrive supplier names
supplier_names = demand_data['TOTAL'].dropna().tolist()
supplier_names.remove('SUM')
# Retrieve production Backlog
production_backlog = demand_data['Unnamed: 3'][0]
stock_at_hand = demand_data['Unnamed: 3'][1]
# Remove unecessary columns and lines before transpose
demand_data.drop(['FG-AA123456', 'TOTAL' , 'Unnamed: 1' , 'Unnamed: 3'], inplace=True, axis=1)
# Transpose table
demand_data = demand_data.T.reset_index(drop=True)
# Get when the supply rows starts being 0 
for column_number in np.arange(1,len(demand_data.columns)):
    inital_row = demand_data[column_number].astype(float).sum() != 0
    if inital_row == False:
        break
# Drop columns 
demand_data.drop(np.arange(column_number,len(demand_data.columns)), inplace=True, axis=1)
# Rename columns
column_names = ['Date', 'Date_Code', 'Demand']
column_names.extend(supplier_names)
demand_data.columns = column_names
# Create Column Confirm Deliveries
demand_data['confirmed_deliveres'] = demand_data[supplier_names].sum(axis=1)
# Create column material balance
demand_data['material_balance'] = np.NaN
# Initial_Value
demand_data['material_balance'][0] = stock_at_hand - production_backlog - demand_data['Demand'][0]
# Generate the rest of the values
for row_number in np.arange(1, len(demand_data)):
   demand_data.loc[row_number, 'material_balance'] = demand_data.loc[row_number-1, 'confirmed_deliveres'] + demand_data.loc[row_number-1, 'material_balance'] - demand_data.loc[row_number, 'Demand']


C:\Users\Yuri_Almeida\AppData\Local\Temp\ipykernel_18892\2452884364.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demand_data['material_balance'][0] = stock_at_hand - production_backlog - demand_data['Demand'][0]


# Graph Display

In [3]:
import bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

In [4]:
p = figure( plot_width=1000, plot_height=800)
p.title.text = 'Cloud_Cover X Num_Trips by Week Days'

p.line(x= demand_data['Date_Code'],
           y=demand_data['material_balance'],
          line_color='lime', line_alpha=0.8, legend_label ='Material Balance',
          muted_color='lime', muted_alpha=0.2)
p.line(x= demand_data['Date_Code'],
           y=demand_data['Demand'],
          line_color='purple', line_alpha=0.8, legend_label ='Demand',
          muted_color='purple', muted_alpha=0.1)
p.line(x=demand_data['Date_Code'],
           y=demand_data['confirmed_deliveres'],
          line_color='blue', alpha=0.8, legend_label ='Supply Deliveres',
          muted_color='blue', muted_alpha=0.1)

p.legend.location = "top_left"
p.legend.click_policy="mute"#"hide"
p.xaxis.axis_label = 'Number of Units'
p.yaxis.axis_label = 'Date_Code'

show(p)